In [1]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open
import xml.etree.ElementTree as ET
import math
p = PorterStemmer()


C:\Users\carlo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Procesamiento de el documento

In [2]:
def process(text):
    doc_nor= text.lower()
    doc_sw = remove_stopwords(doc_nor)
    doc_stem = p.stem_sentence(doc_sw)
    return doc_stem.split()

## Lista de Documentos Procesados

In [15]:
docDict = []
for index in range(1,332):
    number = ""
    if math.floor(index/10) == 0:
        number = "00"+str(index)
    else:
        if math.floor(index/100) == 0:
            number = "0"+str(index)
        else:
            if math.floor(index/1000) == 0:
                number = str(index)
    root = ET.parse('docs-raw-texts/wes2015.d'+number+'.naf').getroot()
    title = root.find('nafHeader').find('fileDesc').attrib['title']
    content = root.find('raw').text
    docDict.append(process(title))
print(docDict[35])

['socrat', 'socrat', 'method']


## Generar Diccionario

In [4]:
dictionary = corpora.Dictionary(docDict)
dictionary.save('clase3.dict')
print(dictionary)

Dictionary(1078 unique tokens: ['beaumont', 'digest', 'human', 'william', 'adventur']...)


C:\Users\carlo\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Generar Corpus

In [5]:
class MyCorpus(object):
    def __iter__(self):
        for index in range(1,332):
            number = ""
            if math.floor(index/10) == 0:
                number = "00"+str(index)
            else:
                if math.floor(index/100) == 0:
                    number = "0"+str(index)
                else:
                    if math.floor(index/1000) == 0:
                        number = str(index)
            root = ET.parse('docs-raw-texts/wes2015.d'+number+'.naf').getroot()
            title = root.find('nafHeader').find('fileDesc').attrib['title']
            content = root.find('raw').text
            yield dictionary.doc2bow(process(title))

corpus_memory_friendly = MyCorpus()#Todos mis documentos ahora estan representados como una bolsa de palabras
#Almaceno mi corpus
corpora.MmCorpus.serialize('corpus.mm', corpus_memory_friendly)  

## Cargar el Corpus

In [6]:
#Cargo mi corpus
corpus = corpora.MmCorpus('corpus.mm')
print(corpus) #No lo carga en memoria
#Para leer la representacion de bolda de palabras resultante de cada documento en el corpus
for doc in corpus:
    print(doc)

MmCorpus(331 documents, 1078 features, 1493 non-zero entries)
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0)]
[(4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0)]
[(10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0)]
[(14, 1.0), (15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]
[(21, 1.0), (22, 1.0), (23, 1.0), (24, 1.0), (25, 1.0)]
[(26, 1.0), (27, 1.0), (28, 1.0), (29, 1.0)]
[(30, 1.0), (31, 1.0), (32, 1.0), (33, 1.0)]
[(34, 1.0), (35, 1.0), (36, 1.0), (37, 1.0), (38, 1.0)]
[(39, 1.0), (40, 1.0), (41, 1.0), (42, 1.0), (43, 1.0), (44, 1.0), (45, 1.0)]
[(46, 1.0), (47, 1.0), (48, 1.0), (49, 1.0), (50, 1.0)]
[(51, 1.0), (52, 1.0), (53, 1.0), (54, 1.0), (55, 1.0)]
[(56, 1.0), (57, 1.0), (58, 1.0), (59, 1.0)]
[(44, 1.0), (45, 1.0), (60, 1.0), (61, 1.0), (62, 1.0), (63, 1.0), (64, 1.0), (65, 1.0)]
[(66, 1.0), (67, 1.0), (68, 1.0), (69, 1.0), (70, 1.0)]
[(3, 1.0), (71, 1.0), (72, 1.0), (73, 1.0)]
[(74, 1.0), (75, 1.0), (76, 1.0)]
[(77, 1.0), (78, 1.0), (79, 2.0)]
[(80, 1.0), (81, 1.

## Cargar de la memoria el Corpus y el Diccionario

In [7]:
dictionary = corpora.Dictionary.load('clase3.dict')
corpus = corpora.MmCorpus('corpus.mm')

## TF-IDF

In [8]:
tfidf = models.TfidfModel(corpus) 

## Construir la Matriz

In [9]:
index = similarities.MatrixSimilarity(tfidf[corpus])
index.save('clase3tfidf.index')

In [10]:
index = similarities.MatrixSimilarity.load('clase3tfidf.index')

## Construir el query

In [13]:
for indice in range(1,36):
    number = ""
    if math.floor(indice/10) == 0:
        number = "0"+str(indice)
    else:
        if math.floor(indice/100) == 0:
            number = str(indice)
    root_query = ET.parse('queries-raw-texts/wes2015.q'+number+'.naf').getroot()
    query = root_query.find('raw').text
    # Query a Bag of Words
    query_doc_bow = dictionary.doc2bow(process(query))
    print(process(query))
    #Simillitudes
    sims = index[tfidf[query_doc_bow]]
    lsims = list(enumerate(sims))
    print("Query "+ str(indice))
    for sim in lsims:
        if(sim[1]!=0):
            print(sim)

['fabric', 'music', 'instrument']
Query 1
(214, 0.45499486)
['famou', 'german', 'poetri']
Query 2
(22, 0.2069323)
(50, 0.17898658)
(148, 0.40139148)
(186, 0.30821532)
(209, 0.2508483)
(286, 0.29384825)
(319, 0.22720699)
(321, 0.25321454)
['romantic']
Query 3
['univers', 'edinburgh', 'research']
Query 4
(129, 0.3276608)
(220, 0.28172344)
['woman', 'won', 'nobel', 'prize']
Query 5
(144, 0.70710677)
['bridg', 'construct']
Query 6
(24, 0.41553405)
(25, 0.40228185)
(296, 0.47389293)
(328, 0.39179713)
['walk', 'fame', 'star']
Query 7
(97, 0.28040847)
(116, 0.27553883)
(145, 0.37422323)
(288, 0.22991851)
['scientist', 'work', 'atom', 'bomb']
Query 8
(4, 0.2014606)
(234, 0.15850872)
(250, 0.59480214)
(264, 0.19040002)
(280, 0.18997896)
(289, 0.17786495)
(291, 0.26558533)
(292, 0.25764376)
(308, 0.21546707)
['invent', 'internet']
Query 9
(51, 0.34877127)
(176, 0.38985527)
(198, 0.46729457)
(322, 0.2642765)
['earli', 'telecommun', 'method']
Query 10
(29, 0.34780452)
(33, 0.49730906)
(35, 0.37564